In [1]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from preprocessor import Preprocessor
from evaluator import ModelEvaluator
import pandas as pd

In [2]:
'''
Изготовим полиномиальные фичи из топ 20 фичей по корреляции с таргетом, но не более 200 штук
'''

'\nИзготовим полиномиальные фичи из топ 20 фичей по корреляции с таргетом, но не более 200 штук\n'

In [3]:
df_train = pd.read_csv("C:/Users/Роман/Chepenkov_dz2/application_train.csv")
df_test = pd.read_csv("C:/Users/Роман/Chepenkov_dz2/application_test.csv")

In [6]:
# Глобальное состояние
POLY_FEATURES = {
    'columns': None,
    'numeric_cols': None
}

def hypothesis_7(df):
    global POLY_FEATURES

    train_target = df_train['TARGET']
    
    df = df.copy()
    numeric_cols = df.select_dtypes(include=['number']).columns.tolist()
    
    if POLY_FEATURES['columns'] is None:
        # Тренировочный режим
        if len(numeric_cols) > 20:
            corr = df[numeric_cols].corrwith(train_target).abs()
            numeric_cols = corr.nlargest(20).index.tolist()
        
        poly = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)
        features = poly.fit_transform(df[numeric_cols])
        cols = poly.get_feature_names_out(numeric_cols)[:1000]
        
        POLY_FEATURES = {
            'columns': cols,
            'numeric_cols': numeric_cols
        }
        
        poly_df = pd.DataFrame(features[:, :1000], columns=cols, index=df.index)
    else:
        # Тестовый режим
        poly = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)
        poly.fit(df[POLY_FEATURES['numeric_cols']])
        features = poly.transform(df[POLY_FEATURES['numeric_cols']])
        
        poly_df = pd.DataFrame(
            data=features[:, :len(POLY_FEATURES['columns'])], 
            columns=POLY_FEATURES['columns'],
            index=df.index
        )
    
    return pd.concat([df, poly_df], axis=1)

In [7]:
preprocessor = Preprocessor(custom_steps=[hypothesis_7])

X_train_processed = preprocessor.transform(df_train.drop(columns=['TARGET', 'SK_ID_CURR']))
num_cols = X_train_processed.select_dtypes(include='number').columns
X_train_processed = X_train_processed[num_cols]

X_test_processed = preprocessor.transform(df_test.drop(columns=['SK_ID_CURR']))
X_test_processed = X_test_processed[num_cols]

In [8]:
evaluator = ModelEvaluator(LogisticRegression(max_iter=100))
roc_auc = evaluator.evaluate(X_train_processed, df_train['TARGET'])
print(f"ROC-AUC train: {roc_auc:.5f}")

evaluator.save_test_predictions(
    X_test=X_test_processed,
    ids=df_test['SK_ID_CURR'],
    filename='test_predictions_hyp_7.csv'
)
#ROC-AUC train: 0.60054

#ROC-AUC test: 0.59334
#ROC-AUC test baseline: 0.61204
#ROC-AUC test surplus: -0,0187

C:\Users\Роман\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


ROC-AUC train: 0.60054
